In [ ]:
#draft for helper functions

In [ ]:
# import necessary libs
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.optim as optim
import time
from PIL import Image
import torchvision
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data.sampler import SubsetRandomSampler
# check, sanity = 0, 0 #global parameter used for testing input into dataloader
plot = False

In [ ]:
def get_accuracy(model, train_loader,val_loader,train=False):
    
    start_time = time.time()

    if train:
        data = train_loader
    else:
        data = val_loader

    correct = 0
    total = 0
    for imgs, labels in data:
        # #############################################
        # #To Enable GPU Usage
        # if use_cuda and torch.cuda.is_available():
        #   imgs = imgs.cuda()
        #   labels = labels.cuda()
        # #############################################
        output = model(imgs)
        
        #select index with maximum prediction score
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += imgs.shape[0]
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("get_accuracy takes: {:.2f} seconds".format(elapsed_time))
    
    return correct / total

In [ ]:
# updated version of train function with isolated test_loader

def train_isoTest(model, batch_size=64, lr=0.01,num_epochs=30):

    global train_loader
    global val_loader
  
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(),
                                 lr=learning_rate, 
                                 weight_decay=1e-5)

    iters, losses, train_acc, val_acc, epochs = [], [], [], [], []
    # training
    n,k = 0,0 # the number of iterations
    for epoch in range(num_epochs):
        for imgs, labels in iter(train_loader):
            # #############################################
            # #To Enable GPU Usage
            # if use_cuda and torch.cuda.is_available():
            #   imgs = imgs.cuda()
            #   labels = labels.cuda()
            # #############################################
            out = model(imgs)             # forward pass
            loss = criterion(out, labels) # compute the total loss
            loss.backward()               # backward pass (compute parameter updates)
            optimizer.step()              # make the updates for each parameter
            optimizer.zero_grad()         # a clean up step for PyTorch

            #save the current training information
            iters.append(n)
            losses.append(float(loss)/batch_size)             # compute *average* loss
            n += 1

        k+=1
        epochs.append(k)
        train_acc.append(get_accuracy(model, train_loader,val_loader, train=True)) # compute training accuracy 
        val_acc.append(get_accuracy(model, train_loader,val_loader, train=False))  # compute validation accuracy
        print(("Epoch {}: loss: {}, Train acc: {} |"+
               "Validation acc: {}").format(
                   epoch + 1,
                   losses[epoch],
                   train_acc[epoch],
                   val_acc[epoch]))

    global plot
    if plot == True:
        # plotting
        plt.title("Training Curve")
        plt.plot(iters, losses, label="Train")
        plt.xlabel("Iterations")
        plt.ylabel("Loss")
        plt.show()

        plt.title("Training Curve")
        plt.plot(epochs, train_acc, label="Train")
        plt.plot(epochs, val_acc, label="Validation")
        plt.xlabel("Epochs")
        plt.ylabel("Training Accuracy")
        plt.legend(loc='best')
        plt.show()
        #global plot
        plot = False
    

    print("Final Training Accuracy: {}".format(train_acc[-1]))
    print("Final Validation Accuracy: {}".format(val_acc[-1]))
